In [1]:
import itertools
import os
import sys
import random
import math
import dill as pickle
import csv
import json
import multiprocessing
import numpy as np
from collections import Counter
from collections import defaultdict
from pathlib import Path
from random import shuffle
import re
import string

In [2]:
def save_model(file_path,model):
    file = open(file_path,"wb")
    pickle.dump(model,file,protocol=2)
    file.close()

def load_model(file_path):
    file = open(file_path,"rb")
    model = pickle.load(file)
    file.close()
    return model

def write_in_file(filename,data):
    ##data is list of strings
    ##filename is file_path
    print("Writing content in file....")
    fd = open(filename,"w",encoding="utf-8")
    for sent in data:
        fd.write(sent+"\n")
    fd.close()
    print("Done!!")

In [13]:
cwd = os.getcwd()
home=str(Path.home())
csnli_path=os.path.join(home,"csnli")
#code_source=os.path.join(cwd,"python_itunes")
#lyrics_data_path=os.path.join(code_source,"data/lyrics")
lyrics_data_path = cwd+"/python_itunes/Genre-Classification-of-Hindi-Songs/backup/only_movies_songs/data/lyrics"
genre_dirs = sorted(os.listdir(lyrics_data_path))
delims = load_model(os.path.join(csnli_path,"delim.pkl"))
print(delims)
print(genre_dirs)

["'", '>', '5', '`', 'ê', '—', '�', '–', '☻', '?', '0', '*', '%', '3', ',', '\n', ';', ']', '9', '4', '(', '=', '1', '2', '^', '&', '☺', '…', '@', '6', 'é', '🙂', '☓', '‘', ':', '8', '+', '}', '_', '[', '!', '<', '|', '"', '.', '{', '~', '“', '/', '\\', 'â', '7', '”', '♫', '×', '$', '#', 'ä', ')']
['bhajan', 'bhangra', 'birthday', 'classical', 'dance', 'dream', 'engagement', 'festivals', 'folk', 'friendship', 'funny', 'happy', 'hip-pop', 'item-numbers', 'kids', 'love', 'marriage', 'masti', 'motivational', 'mujra', 'party', 'patriotic', 'philosophical', 'playful', 'pop', 'qawwali', 'rain', 'religious', 'rock', 'sad', 'sensual', 'sharaabi', 'sufi']


In [14]:
def tokenise(line):
    line = line.lower()
    for delim in delims:
        line = line.replace(delim,"")
    return line

def read_genre_data(genre,tracker,is_tokenise=True):
    corpus=[]
    for file in sorted(os.listdir(os.path.join(lyrics_data_path,genre))):
        doc=[]
        file_path = os.path.join(os.path.join(lyrics_data_path,genre),file)
        if os.stat(file_path).st_size==0:
            #print(genre,file_path)
            continue
        lyrics = open(file_path,"r").readlines()
        for line in lyrics:
            if line.split("\n")[0].strip():
                input_=line.split("\n")[0]
                if is_tokenise:
                    input_=tokenise(input_).strip()
                if len(input_.split())>0:
                    doc.append(input_)
        if doc not in corpus:
            if doc not in tracker:
                tracker.append(doc)
                corpus.append(doc)
    return corpus,tracker
def sanity_checks(data):
    for genre in data:
        temp=[]
        for song in data[genre]:
            if tuple(song) not in temp:
                temp.append(tuple(song))
            else:
                raise Exception("Duplicate song in data!\n")
                #print(genre,song,"\n\n")
    temp=defaultdict(list)
    for genre in data:
        for song in data[genre]:
            temp[tuple(song)].append(genre)
    for song in temp:
        if len(temp[song])>1:
            #print(song,len(temp[song]),temp[song],"\n\n")
            raise Exception("Duplicate songs in same genre!\n")        

In [15]:
## data is dictionary where key is 'genre' mapped to list of lists (Each list correpsonds to 'song lyrics')
data={}
tracker=[]
for genre in genre_dirs:
    data[genre],tracker=read_genre_data(genre,tracker)

In [16]:
sanity_checks(data)

In [17]:
##Songs in data
cnt=0
for genre in data:
    cnt+=len(data[genre])
print("Total songs in data: ",cnt,"\n")

Total songs in data:  28833 



In [18]:
#save_model("pickled_data/genre_data_map.pkl",data)
data=load_model("pickled_data/genre_data_map.pkl")

In [19]:
##Songs in data
cnt=0
for genre in data:
    cnt+=len(data[genre])
print("Total songs in data: ",cnt,"\n")
print(data[genre_dirs[0]][:5])

Total songs in data:  28823 

[['prabhu ji prabhu ji', 'tum rakho laaj hamari', 'prabhu ji prabhu ji', 'tum rakho laaj hamari', 'man ki baat chhupi nahi sabse', 'man ki baat chhupi nahi sabse', 'joban dalke sari', 'rakho laaj hamari', 'prabhu ji prabhu ji tum', 'kya vidhata jaal bichhaya', 'aur biraja ka taro mein aaya', 'man panchhi usme samata', 'kismat mein tha dukho ne jana', 'is bandhan se tum hi chhudao', 'is bandhan se tum hi chhudao', 'pyaare krishan murari', 'rakho laaj hamari', 'prabhu ji prabhu ji tum', 'kaisa dekh rahi hoon sapna', 'sapno ki kya hota apna', 'kaisa dekh rahi hoon sapna', 'sapno ki kya hota apna', 'jo kuch hua meri marji', 'mujh ko sone na de', 'rakho laaz hamari', 'prabhu ji prabhu ji tum', 'door baho na aao aao', 'tumse dheer bandao aao', 'jeevan mein toofan utha hai', 'prabhu ji prabhu ji', 'jeevan mein toofan utha hai', 'aur aankho abhi yaad', 'rakho laaz hamari', 'prabhu ji prabhu ji', 'tum rakho laaj hamari'], ['kunjan ban chhaadi he', 'madho kahaan jaa

In [28]:
print(genre_dirs)
#WIF :- Western-Indian Fusion Music
merged_classes={"indian_wedding":["marriage","engagement"],
                "joyful":["playful","funny","masti"],
                "sensual":["item-numbers","sensual"],
                "religious":["religious","bhajan","sufi","qawwali"],
                "WIF":["pop","rock","hip-pop","party","dance"],
                "folk":["folk","bhangra"],
                "other":["kids","rain","sharaabi","dream"]
               }
main_classes={"theme":["friendship","birthday","festivals","patriotic","indian_wedding","mujra"],
              "mood":["happy","sad","joyful","sensual","love","motivational","philosophical"],
              "genre":["classical","religious","folk","WIF"],
              "other_class":["other"]
             }

['bhajan', 'bhangra', 'birthday', 'classical', 'dance', 'dream', 'engagement', 'festivals', 'folk', 'friendship', 'funny', 'happy', 'hip-pop', 'item-numbers', 'kids', 'love', 'marriage', 'masti', 'motivational', 'mujra', 'party', 'patriotic', 'philosophical', 'playful', 'pop', 'qawwali', 'rain', 'religious', 'rock', 'sad', 'sensual', 'sharaabi', 'sufi']


In [30]:
'''main_data=defaultdict(lambda: defaultdict(list))
for label in main_classes:
    for sublabel in main_classes[label]:
        if sublabel not in merged_classes:
            main_data[label][sublabel]=data[sublabel]
        else:
            for subsublabel in merged_classes[sublabel]:
                main_data[label][sublabel].extend(data[subsublabel])'''

'main_data=defaultdict(lambda: defaultdict(list))\nfor label in main_classes:\n    for sublabel in main_classes[label]:\n        if sublabel not in merged_classes:\n            main_data[label][sublabel]=data[sublabel]\n        else:\n            for subsublabel in merged_classes[sublabel]:\n                main_data[label][sublabel].extend(data[subsublabel])'

In [32]:
#save_model("pickled_data/songs_main_data.pkl",main_data)
main_data = load_model("pickled_data/songs_main_data.pkl")

In [33]:
for label in main_data:
    for sublabel in main_data[label]:
        print(label,sublabel,len(main_data[label][sublabel]))

theme friendship 89
theme birthday 33
theme festivals 212
theme patriotic 291
theme indian_wedding 390
theme mujra 340
mood happy 5408
mood sad 4583
mood joyful 5973
mood sensual 834
mood love 3668
mood motivational 458
mood philosophical 472
genre classical 316
genre religious 613
genre folk 219
genre WIF 4160
other_class other 764


In [35]:
t=0
for label in main_data:
    for sublabel in main_data[label]:
        t+=len(main_data[label][sublabel])
t##total songs

28823